In [2]:
from rich.console import Console
from rich.json import JSON
from rich.markdown import Markdown
console = Console()

In [3]:
from datasets import load_dataset,Dataset
import json
import os
import shutil

/dfs/data/uv-venv/modelscope/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
graphsyn_path="../ToolMind/graph_syn_datasets/graphsyn.jsonl"

In [5]:
cache_path = '/dfs/data/.cache/huggingface/datasets/generator'
if os.path.exists(cache_path):
    print(f"正在清理缓存: {cache_path}")
    try:
        shutil.rmtree(cache_path)
    except Exception as e:
        print(f"缓存清理失败 (通常不影响运行): {e}")

In [6]:
def robust_data_generator(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                row = json.loads(line)
                
                # --- 深度清洗逻辑 ---
                if "conversations" in row:
                    for conv in row["conversations"]:
                        # 1. 确保 content 是字符串 (有些数据集里 content 可能是 null)
                        if conv.get("content") is None:
                            conv["content"] = ""
                            
                        # 2. 处理 tool_calls
                        if "tool_calls" in conv:
                            tc_data = conv["tool_calls"]
                            
                            # 情况 A: tool_calls 是 None -> 转为空列表
                            if tc_data is None:
                                conv["tool_calls"] = []
                                continue
                                
                            # 情况 B: tool_calls 是列表 -> 检查内部
                            if isinstance(tc_data, list):
                                for tc in tc_data:
                                    # 检查 function.arguments
                                    if "function" in tc:
                                        func = tc["function"]
                                        args = func.get("arguments")
                                        
                                        # === 核心修复 ===
                                        # 无论 arguments 是字典还是字符串，统一转为字符串
                                        # 这样可以避免 "cannot mix struct and non-struct"
                                        if isinstance(args, dict):
                                            func["arguments"] = json.dumps(args, ensure_ascii=False)
                                        elif args is None:
                                            func["arguments"] = "{}"
                                        # 如果已经是 string，保持不变
                            
                yield row
            except json.JSONDecodeError:
                continue

In [7]:
ds = Dataset.from_generator(robust_data_generator, gen_kwargs={"file_path": graphsyn_path})

console.print(ds)

Generating train split: 999 examples [00:00, 4620.23 examples/s]


OSError: [Errno 39] Directory not empty: '/dfs/data/.cache/huggingface/datasets/generator/default-554e5d0fbe88b8bb/0.0.0.incomplete'